In [33]:
import numpy as np
import torch
import torch.nn as nn

import sys
sys.path.append('/home/jhultman/Projects/tmp/PV-RCNN/Pointnet2.PyTorch/')

from pointnet2.pointnet2_modules import PointnetSAModuleMSG
from pointnet2.pointnet2_utils import furthest_point_sample, gather_operation
import pointnet2.pytorch_utils as pt_utils

In [34]:
net = PointnetSAModuleMSG(
    npoint=-1, radii=[0.1, 0.5], nsamples=[16, 32],
    mlps=[[1, 16, 32], [1, 32, 64]], use_xyz=True,
).cuda()

In [35]:
points = np.fromfile('./sample.bin', dtype=np.float32).reshape(-1, 4)
points = torch.from_numpy(points).cuda().unsqueeze(0)
points = points[:, ::10]

In [36]:
xyz, features = torch.split(points, [3, 1], dim=-1)
xyz = xyz.contiguous()
features = features.permute(0, 2, 1).contiguous()
indices = furthest_point_sample(xyz, 2048).squeeze(0).long()
keypoints = points[:, indices]
keypoints_xyz, keypoints_features = torch.split(keypoints, [3, 1], dim=-1)
_, out = net(xyz, features, keypoints_xyz.contiguous())

In [37]:
out.shape

torch.Size([1, 96, 2048])